## 4.3 JsonOutputParser

<div style="text-align: right"> Initial issue : 2025.05.02 </div>
<div style="text-align: right"> last update : 2025.05.02 </div>

사용자가 원하는 JSON 스키마를 지정할 수 있게 해주는 도구. 
LLM이 데이터를 조회하고 결과를 도출할 때, 지정된 스키마에 맞게 JSON 형식으로 데이터를 반환할 수 있도록 설됨

사용자가 원하는 형태의 JSON을 생성하기 위해서는, 모델의 용량(예: 인텔리전스)이 충분히 커야함. 
예를 들어, llama-70B 모델은 llama-8B 모델보다 더 큰 용량을 가지고 있어 보다 복잡한 데이터를 처리하는 데 유리함

### 참고: JSON
- `JSON (JavaScript Object Notation)` 은 데이터를 저장하고 구조적으로 전달하기 위해 사용되는 경량의 데이터 교환 포맷. 
- 웹 개발에서 매우 중요한 역할을 하며, 서버와 클라이언트 간의 통신을 위해 널리 사용됨. 
- JSON은 읽기 쉽고, 기계가 파싱하고 생성하기 쉬운 텍스트 기반임

### JSON의 기본 구조
- JSON 데이터는 이름(키)과 값의 쌍으로 이루어져 있음. 
- 여기서 "이름"은 문자열이고, "값"은 다양한 데이터 유형일 수 있음. 
- JSON은 두 가지 기본 구조를 가짐
    - 객체: 중괄호 {}로 둘러싸인 키-값 쌍의 집합. 각 키는 콜론 :을 사용하여 해당하는 값과 연결되며, 여러 키-값 쌍은 쉼표 ,로 구분됨.
    - 배열: 대괄호 []로 둘러싸인 값의 순서 있는 목록. 배열 내의 값은 쉼표 ,로 구분됨.

```json
{
  "name": "John Doe",
  "age": 30,
  "is_student": false,
  "skills": ["Java", "Python", "JavaScript"],
  "address": {
    "street": "123 Main St",
    "city": "Anytown"
  }
}
```


In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

In [3]:
model = ChatOpenAI(temperature=0, model_name="gpt-4o")

원하는 출력 구조 정의

In [4]:
class Topic(BaseModel):
    description: str = Field(description="주제에 대한 간결한 설명")
    hashtags: str = Field(description="해시태그 형식의 키워드(2개 이상)")

`JsonOutputParser`를 사용하여 파서를 설정

In [5]:
# 질의 작성
question = "지구 온난화의 심각성 대해 알려주세요."

# 파서를 설정하고 프롬프트 템플릿에 지시사항을 주입합니다.
parser = JsonOutputParser(pydantic_object=Topic)
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"description": {"description": "주제에 대한 간결한 설명", "title": "Description", "type": "string"}, "hashtags": {"description": "해시태그 형식의 키워드(2개 이상)", "title": "Hashtags", "type": "string"}}, "required": ["description", "hashtags"]}
```


프롬프트 정의

In [6]:
# 프롬프트 템플릿을 설정합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요."),
        ("user", "#Format: {format_instructions}\n\n#Question: {question}"),
    ]
)

prompt = prompt.partial(format_instructions=parser.get_format_instructions())

In [7]:
print(prompt.format(format_instructions="test_instruction", question="test_input"))

System: 당신은 친절한 AI 어시스턴트 입니다. 질문에 간결하게 답변하세요.
Human: #Format: test_instruction

#Question: test_input


Chain 생성

In [8]:
# 체인을 구성합니다.
chain = prompt | model | parser

답변 생성

In [9]:
answer = chain.invoke({"question": question})

In [10]:
type(answer)

dict

In [11]:
answer

{'description': '지구 온난화는 지구의 평균 기온이 상승하는 현상으로, 주로 인간 활동에 의해 발생하는 온실가스 배출이 주요 원인입니다. 이는 극지방의 빙하가 녹고 해수면이 상승하며, 기후 패턴이 변화하는 등 다양한 환경적 문제를 초래합니다.',
 'hashtags': '#지구온난화 #기후변화 #환경문제'}